In [ ]:
# def subscribe(tfm):
#     old_call = tfm._call
#     def _inner(f):
#         def _call(self, fn, x, **kwargs):
#             res = old_call(self, fn, x, **kwargs)
#             if res is not x: f(res)
#             return res
#         tfm._call = _call
#         return f
#     return _inner

* Modify typedispatch to accept classes type?

Create subclass for string  
Subscribe on class instead of instance  
Use type dispatch to subscribe non transform and functions
  * Needs to patch on variable type, that will be passed by subscribe  
      * subscribe(Tokenizer)

In [ ]:
from fastai2.basics import *

In [ ]:
class AttrProxy(GetAttr):
    def __init__(self, default): self.default = default

In [ ]:
def _get_proxy(x):
    if x.__class__.__module__ != 'builtins': raise ValueError('Use only with builtins')
    name = 'Proxy' + x.__class__.__name__.capitalize()
    return type(name, (x.__class__,), {})(x)

In [ ]:
# class MyStr(str): pass
# class MyInt(int): pass

In [ ]:
def _maintain_labels(old, new):
    if hasattr(old, 'labels'): new.labels = old.labels

In [ ]:
_old_call = Transform._call
@patch
def _call(self:Transform, fn, x, **kwargs):
    res = _old_call(self, fn, x, **kwargs)
    _maintain_labels(x, res)
    return res

In [ ]:
def _add_attr(obj, name, value):
    try:                   setattr(obj, name, getattr(obj,'labels',[])); return obj
    # It's not possible to set attributes on builtin types, so we wrap with a proxy
    except AttributeError: return _add_attr(_get_proxy(obj), name, value)

In [ ]:
def _add_label(f):
    def _inner(x):
        label = f(x)
        x = _add_attr(x, 'labels', [])
        x.labels.append(label)
        return x
    return _inner

In [ ]:
# @typedispatch
# def subscribe(f):
#     old_call = f.__call__
#     def _inner(f_):
#         def _call(x, **kwargs):
#             res = old_call(x, **kwargs)
#             _maintain_labels(x, res) # Little difference
#             if res is not x: f_(res) # I may also want to handle this case differently
#             return res
#         f.__call__ = _call
#         return f_
#     return _inner

In [ ]:
@typedispatch
def subscribe(tfm):
    old_call = tfm.__call__
    def _inner(f):
        def _call(self, x, **kwargs):
            res = old_call(self, x, **kwargs)
            _maintain_labels(x, res)
            if res is not x: res = f(res)
            return res
        tfm.__call__ = _call
        return f
    return _inner

In [ ]:
def labeller(tfm):
    def _inner(f):
        return subscribe(tfm)(_add_label(f))
    return _inner

In [ ]:
# Does not work with wrapper style
# @Transform
# def neg(x:Tensor): return -x
# @Transform
# def add_greeting(x:str): return 'hello ' + x

In [ ]:
class Neg(Transform):
    def encodes(self, x:Tensor): return -x

In [ ]:
def func(x): return x+3

In [ ]:
# @labeller(neg)
# def labeller_cat1(x): return CAT1
# @labeller(neg)
# def labeller_cat2(x): return CAT2

In [ ]:
# @labeller(add_greeting)
# def labeller_greeting(x): return CAT1

In [ ]:
CAT1,CAT2 = 'cat1','cat2'

In [ ]:
@labeller(Neg)
def func1(x): return CAT2

In [ ]:
# x = tensor(2)
# tfms = Pipeline([Neg])
# res = tfms(x)

In [ ]:
# res.labels

In [ ]:
# @Transform
# def str2int(x:MyStr): return MyInt(float(x))
# @Transform
# def int2tensor(x:MyInt): return tensor(x)

In [ ]:
# @labeller(str2int)
# def func1(x): return 'str2int'
# @labeller(int2tensor)
# def func2(x):
#     if x>2: return 'int2tensor'

In [ ]:
# pipe = Pipeline([str2int, int2tensor])

In [ ]:
# res = pipe(MyStr('1.2'))
# res.labels

## IMDB tests

In [ ]:
from fastai2.text.all import *

In [ ]:
source = untar_data(URLs.IMDB_SAMPLE)

In [ ]:
df = pd.read_csv(source/'texts.csv')

In [ ]:
splits = ColSplitter('is_valid')(df)

In [ ]:
df['original'] = df['text']

In [ ]:
col_reader = mk_transform(ColReader('original'))
tkzer = Tokenizer.from_df('text')
nmzer = Numericalize()

In [ ]:
POS,NEG = 'positive','negative'

In [ ]:
@labeller(Tokenizer)
def test(x):
    print('tokenizer')
    return POS

In [ ]:
@labeller(ColReader)
def liked(x):
    print('col')
    return NEG

In [ ]:
dset = Datasets(df, tfms=[[ColReader('original'), Tokenizer.from_df('text'), Numericalize()]])

col
tokenizer


In [ ]:
res = dset[0][0]

col
tokenizer


In [ ]:
res.labels

['negative', 'positive']